---
# --- Day 17: Pyroclastic Flow ---
---

In [ ]:
import numpy as np

## Load data

In [ ]:
full_puzzle_data = False

In [ ]:
file_suffix = "" if full_puzzle_data else "_test"
with open(f"data/day17_input{file_suffix}.txt", "r") as f:
    data = f.read().splitlines()

## --- Part One ---

## --- Part Two ---